# Task2 Crawl Pull Requests

## Tools used
For this task, I will use requests to get the html content of the pull request page, and extract the information using beautifulsoup and regular expression.

In [30]:
import requests
import re

In [11]:
from bs4 import BeautifulSoup as bs

## Feature extracting functions

In [4]:
def get_title(soup):
    try:
        return str(soup.find('span',{'class': lambda x: x and 'js-issue-title' in x}).text).replace('\n','')
    except:
        return None

In [2]:
def get_labels(soup):
    labels = []
    for i in soup.find_all('div',{'class': lambda x: x and "js-issue-labels" in x}):
        for j in i.find_all('a'):
            labels.append(j.text)
    return labels

In [3]:
def get_reviewers_assignees(soup):
    reviewers = []
    assignees = []
    for i in soup.find_all('form'):
        if i.has_attr('aria-label'):
            if i['aria-label']=='Select reviewers':
                for j in i.find_all('span'):
                    if j.has_attr('data-assignee-name'):
                        reviewers.append(j['data-assignee-name'])
            if i['aria-label']=='Select assignees':
                for j in i.find_all('span'):
                    if j.has_attr('data-assignee-name'):
                        assignees.append(j['data-assignee-name'])
    return reviewers,assignees

Because the minimized comments are using javascript, only the unminimized comments are extracted here.

Extracting minimized comments is possible if we use webdriver, but it will use too much time and computing resource.

In [58]:
def get_unminimized_comments(soup):
    comments = []
    try:
        for i in soup.find_all(None,{'class': lambda x: x and "unminimized-comment" in x}):
            author = i.find(None,{'class': lambda x:x and 'author' in x}).text
            comment = ''
            for j in i.find_all('p'):
                comment+=j.text+' '
            comments.append(author+": "+comment)
        return comments
    except:
        return []

In [8]:
def get_opened_time(soup):
    try:
        for i in soup.find_all(None,{'class': lambda x: x and 'js-discussion ' in x}):
            return i.find('relative-time')["datetime"]
    except:
        return None

In [9]:
def get_end_time(soup):
    try:
        return re.search('closed this.*?datetime="(.*?)".*?</relative-time>',str(soup),re.DOTALL).group(1)
    except:
        return None

In [46]:
def get_n_commits(soup):
    try:
        return int(soup.find(None,{'id': 'commits_tab_counter'}).text)
    except:
        return 0

In [47]:
def get_n_changed_files(soup):
    try:
        return int(soup.find(None,{'id': 'files_tab_counter'}).text)
    except:
        return 0

In [48]:
def get_n_participants(soup):
    try:
        return int(re.search('\d+',soup.find(None,{'class':'participation'}).div.text).group())
    except:
        return 0

## Crawling

In [59]:
import pandas as pd
import numpy as np
import multiprocessing
thelock = multiprocessing.Lock()



def crawl_pull(x):
    global data
    print(str(x)+ ' started\n',end='')
    a = requests.get("https://github.com/google/guava/pull/"+str(x))
    i = 0
    while not(a.status_code in [200]):
        if i>10:
            return
        i+=1
        print("retry {}\n".format(x),end='')
        a = requests.get("https://github.com/google/guava/pull/"+str(x))
    soup = bs(a.text,'html.parser')
    myjson = {}
    myjson['id'] = x
    myjson['title'] = get_title(soup)
    myjson['reviewers'],myjson['assignees'] = get_reviewers_assignees(soup)
    myjson['labels'] = get_labels(soup)
    myjson['comments'] = get_unminimized_comments(soup)
    myjson['opened_time'] = get_opened_time(soup)
    myjson['closed_time'] = get_end_time(soup)
    myjson['n_commits'] = get_n_commits(soup)
    myjson['n_changed_files'] = get_n_changed_files(soup)
    myjson['n_participants'] = get_n_participants(soup)
    thelock.acquire()
    data = data.append(pd.DataFrame([myjson]))
    thelock.release()
    print(str(x)+" done\n",end='')

In [49]:
data = pd.DataFrame(columns = ['id','title','labels','reviewers','assignees','comments',
                               'opened_time','closed_time','n_commits','n_changed_files','n_participants'])

I wanted to use multiprocessing to crawl the requests, but my VPN doesn't allow me to do so.

Here I will use a for loop to crawl the requests.

In [51]:
for i in range(1,5432):
    crawl_pull(i)

1 started
1 done
2 started
2 done
3 started
3 done
4 started
4 done
5 started
5 done
6 started
6 done
7 started
7 done
8 started
8 done
9 started
9 done
10 started
10 done
11 started
11 done
12 started
12 done
13 started
13 done
14 started
14 done
15 started
15 done
16 started
16 done
17 started
17 done
18 started
18 done
19 started
19 done
20 started
20 done
21 started
21 done
22 started
22 done
23 started
23 done
24 started
24 done
25 started
25 done
26 started
26 done
27 started
27 done
28 started
28 done
29 started
29 done
30 started
30 done
31 started
31 done
32 started
32 done
33 started
33 done
34 started
34 done
35 started
35 done
36 started
36 done
37 started
37 done
38 started
38 done
39 started
39 done
40 started
40 done
41 started
41 done
42 started
42 done
43 started
43 done
44 started
44 done
45 started
45 done
46 started
46 done
47 started
47 done
48 started
48 done
49 started
49 done
50 started
50 done
51 started
51 done
52 started
52 done
53 started
53 done
54 started


401 done
402 started
402 done
403 started
403 done
404 started
404 done
405 started
405 done
406 started
406 done
407 started
407 done
408 started
408 done
409 started
409 done
410 started
410 done
411 started
411 done
412 started
412 done
413 started
413 done
414 started
414 done
415 started
415 done
416 started
416 done
417 started
417 done
418 started
418 done
419 started
419 done
420 started
420 done
421 started
421 done
422 started
422 done
423 started
423 done
424 started
424 done
425 started
425 done
426 started
426 done
427 started
427 done
428 started
428 done
429 started
429 done
430 started
430 done
431 started
431 done
432 started
432 done
433 started
433 done
434 started
434 done
435 started
435 done
436 started
436 done
437 started
437 done
438 started
438 done
439 started
439 done
440 started
440 done
441 started
441 done
442 started
442 done
443 started
443 done
444 started
444 done
445 started
445 done
446 started
446 done
447 started
447 done
448 started
448 done
449 

792 done
793 started
793 done
794 started
794 done
795 started
795 done
796 started
796 done
797 started
797 done
798 started
798 done
799 started
799 done
800 started
800 done
801 started
801 done
802 started
802 done
803 started
803 done
804 started
804 done
805 started
805 done
806 started
806 done
807 started
807 done
808 started
808 done
809 started
809 done
810 started
810 done
811 started
811 done
812 started
812 done
813 started
813 done
814 started
814 done
815 started
815 done
816 started
816 done
817 started
817 done
818 started
818 done
819 started
819 done
820 started
820 done
821 started
821 done
822 started
822 done
823 started
823 done
824 started
824 done
825 started
825 done
826 started
826 done
827 started
827 done
828 started
828 done
829 started
829 done
830 started
830 done
831 started
831 done
832 started
832 done
833 started
833 done
834 started
834 done
835 started
835 done
836 started
836 done
837 started
837 done
838 started
838 done
839 started
839 done
840 

1167 done
1168 started
1168 done
1169 started
1169 done
1170 started
1170 done
1171 started
1171 done
1172 started
1172 done
1173 started
1173 done
1174 started
1174 done
1175 started
1175 done
1176 started
1176 done
1177 started
1177 done
1178 started
1178 done
1179 started
1179 done
1180 started
1180 done
1181 started
1181 done
1182 started
1182 done
1183 started
1183 done
1184 started
1184 done
1185 started
1185 done
1186 started
1186 done
1187 started
1187 done
1188 started
1188 done
1189 started
1189 done
1190 started
1190 done
1191 started
1191 done
1192 started
1192 done
1193 started
1193 done
1194 started
1194 done
1195 started
1195 done
1196 started
1196 done
1197 started
1197 done
1198 started
1198 done
1199 started
1199 done
1200 started
1200 done
1201 started
1201 done
1202 started
1202 done
1203 started
1203 done
1204 started
1204 done
1205 started
1205 done
1206 started
1206 done
1207 started
1207 done
1208 started
1208 done
1209 started
1209 done
1210 started
1210 done
1

1524 done
1525 started
1525 done
1526 started
1526 done
1527 started
1527 done
1528 started
1528 done
1529 started
1529 done
1530 started
1530 done
1531 started
1531 done
1532 started
1532 done
1533 started
1533 done
1534 started
1534 done
1535 started
1535 done
1536 started
1536 done
1537 started
1537 done
1538 started
1538 done
1539 started
1539 done
1540 started
1540 done
1541 started
1541 done
1542 started
1542 done
1543 started
1543 done
1544 started
1544 done
1545 started
1545 done
1546 started
1546 done
1547 started
1547 done
1548 started
1548 done
1549 started
1549 done
1550 started
1550 done
1551 started
1551 done
1552 started
1552 done
1553 started
1553 done
1554 started
1554 done
1555 started
1555 done
1556 started
1556 done
1557 started
1557 done
1558 started
1558 done
1559 started
1559 done
1560 started
1560 done
1561 started
1561 done
1562 started
1562 done
1563 started
1563 done
1564 started
1564 done
1565 started
1565 done
1566 started
1566 done
1567 started
1567 done
1

...

retry 4995
retry 4995
retry 4995
4996 started
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
retry 4996
4997 started
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
retry 4997
4998 started
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
retry 4998
4999 started
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
retry 4999
5000 started
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
retry 5000
5001 started
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
retry 5001
5002 started
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
retry 5002
5003 started
retry 5003
retry

retry 5056
5057 started
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
retry 5057
5058 started
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
retry 5058
5059 started
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
retry 5059
5060 started
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
retry 5060
5061 started
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
retry 5061
5062 started
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
retry 5062
5063 started
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
retry 5063
5064 started
retry 5064
retry 5064
retry 5064
retry

retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
retry 5118
5119 started
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
retry 5119
5120 started
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
retry 5120
5121 started
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
retry 5121
5122 started
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
retry 5122
5123 started
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
retry 5123
5124 started
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
retry 5124
5125 started
retry 5125
retry 5125
retry 5125
retry 5125
retry 5125
retry 5

retry 5179
retry 5179
retry 5179
retry 5179
retry 5179
retry 5179
retry 5179
retry 5179
retry 5179
5180 started
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
retry 5180
5181 started
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
retry 5181
5182 started
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
retry 5182
5183 started
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
retry 5183
5184 started
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
retry 5184
5185 started
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
retry 5185
5186 started
retry 5186
retry 5186
retry 5186
retry 5186
retry 5186
retry 5186
retry 5186
retry 5

retry 5240
retry 5240
retry 5240
retry 5240
retry 5240
retry 5240
retry 5240
5241 started
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
retry 5241
5242 started
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
retry 5242
5243 started
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
retry 5243
5244 started
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
retry 5244
5245 started
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
retry 5245
5246 started
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
retry 5246
5247 started
retry 5247
retry 5247
retry 5247
retry 5247
retry 5247
retry 5247
retry 5247
retry 5247
retry 5247
retry 5

The crawled data:

In [53]:
data

,id,title,labels,reviewers,assignees,comments,opened_time,closed_time,n_commits,n_changed_files,n_participants
0,1,Maven repo,"[status=fixed, type=defect]",[],[],[gissuebot: Original issue created by b.k.oxle...,2014-10-31T17:03:26Z,2014-10-31T18:26:27Z,0,0,1
0,2,`Predicate` missing,"[status=will-not-fix, type=defect]",[],[],[gissuebot: Original issue created by j...@nws...,2014-10-31T17:03:28Z,2014-10-31T18:15:58Z,0,0,1
0,3,Javadoc bug (typo) in com.google.commo...,"[status=fixed, type=defect]",[],[],[gissuebot: Original issue created by ivo.weve...,2014-10-31T17:03:30Z,2014-10-31T18:23:49Z,0,0,1
0,4,Joiner seems to be missing,"[status=duplicate, type=defect]",[],[],[gissuebot: Original issue created by ted.dunn...,2014-10-31T17:03:32Z,2014-10-31T18:16:15Z,0,0,1
0,5,Merge google collections into this pro...,[status=fixed],[],[kevinb9n],[gissuebot: Original issue created by kevinb9n...,2014-10-31T17:03:34Z,2014-10-31T18:19:17Z,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
0,5427,Not a tracked issue- Refactor TriePars...,[cla: no],[],[],[olofeldre: This branch / commit contains a mo...,2021-03-04T09:46:59Z,2021-03-04T09:50:55Z,1,0,1
0,5428,Move Java-7-VM warning from `MoreObjec...,[cla: yes],[],[],[copybara-service: Move Java-7-VM warning from...,2021-03-09T01:39:16Z,2021-03-09T02:01:38Z,1,0,1
0,5429,Issue #2394 - Replace LinkedList with ...,[cla: yes],[falconetpt],[],"[antonlovstrom: #2394 Refactor TrieParser, Lin...",2021-03-10T12:54:01Z,2021-03-11T21:31:51Z,1,0,7
0,5430,Replace LinkedList with ArrayDeque.,[cla: yes],[],[],[copybara-service: Replace LinkedList with Arr...,2021-03-11T21:09:38Z,2021-03-11T21:31:51Z,0,0,0


Saving data into .csv file:

In [54]:
data.to_csv('data.csv')